In [ ]:
import torch
import torch.nn as nn  # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.optim as optim  # For all Optimization algorithms, SGD, Adam, etc.
import torchvision.transforms as transforms  # Transformations we can perform on our dataset
import torchvision
import os
import glob # Helps in file searching using pattern
import pandas as pd
import numpy as np
from torch.utils.data import (
    Dataset,
    DataLoader,
) 




class imagedataset(Dataset):
    def __init__(self,root_dir,transform = None):
        self.root_dir = glob.glob(root_dir)
        self.transform = transform
        self.img, self.labels = self.extraction(self.root_dir)
        
    def __len__(self):
        return len(self.root_dir)
    
    def __getitem__(self, idx):
        return self.img[idx], self.labels[idx]
    
    def extraction(self,file):
        img = []
        labels = []
        for objects in file:
            data = np.load(objects)
            img.append(torch.from_numpy(data[0]))
            labels.append(torch.tensor(data[1]))
        return img, labels
        
import torchvision.transforms as transforms
transformations = transforms.Compose([
    transforms.Resize(255),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dataset = imagedataset(r'content/gdrive/MyDrive/work/*.npy',transformations)
train_size = int(len(dataset)*0.9)
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset= dataset, lengths=[train_size, test_size])

train_loader = DataLoader(dataset=train_dataset, batch_size=32) # set workers if too slow
test_loader = DataLoader(dataset=test_dataset, batch_size=32)

class custom_model(nn.Module):
    def __init__(self, input_dim, output_dim, channels=[2, 4, 2, 1], kernels=[3, 3, 3, 1]):
        super(BasicCNN, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.channels = channels
        
        self.layer1 = nn.Sequential(nn.Conv2d(in_channels=self.input_dim, out_channels=channels[0], kernel_size=kernels[0]),
                                    nn.BatchNorm2d(channels[0]),
                                    nn.ReLU(inplace=True),
                                    nn.MaxPool2d(kernel_size=kernels[0]))
        self.layer2 = nn.Sequential(nn.Conv2d(in_channels=channels[0], out_channels=channels[1], kernel_size=kernels[1]),
                                    nn.BatchNorm2d(channels[1]),
                                    nn.ReLU(inplace=True),
                                    nn.MaxPool2d(kernel_size=kernels[1]))
        self.linear = nn.Sequential(nn.Linear(16, self.output_dim),
                                    nn.ReLU(inplace=True))
    
    def forward(self, x):
        x = x.view(-1, 1, 150, 150)
        x = self.layer1(x)
        x = self.layer2(x)
        x = x.view(x.shape[0], -1)
        x = self.linear(x)
        return x




in_channel = 3
num_classes = 2
learning_rate = 1e-3
batch_size = 32
num_epochs = 10


# Reproducibility
torch.manual_seed(42)
torch.cuda.manual_seed(42)
np.random.seed(42)
random.seed(42)
torch.backends.cudnn.deterministic=True

model = custom_model(input_dim=1, output_dim=1)
model.to(device)
loss_function = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001)

start = time.perf_counter()

for epoch in range(num_epochs):
    losses = []

    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        losses.append(loss.item())

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

    print(f"Cost at epoch {epoch} is {sum(losses)/len(losses)}")




    